In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import random
import datetime as dt

from copy import deepcopy as copy
from client_graphql import Client


In [3]:
url = 'ws://localhost:4001/graphql'
url = 'ws://192.168.39.46:30590/graphql'

s = Client(url, channel='channel-1', debug=False)

In [ ]:
s.write('vector2', 15)

In [ ]:
s.read('foo')

In [ ]:
s.read_histo('foo', timestamp=True)

In [4]:
class Output:
    """
    Helper class
    """
    def __init__(self, name='my-display', max_lines=10):
        self.h = display(display_id=name)
        self.content = ''
        self.mime_type = None
        self.max_lines = max_lines
        self.dic_kind = {'text': 'text/plain'}
    def display(self):
        self.h.display({'text/plain': ''}, raw=True)
    def limit(self, text):
        lines = text.split('\n')
        lines = lines[-self.max_lines:]
        return '\n'.join(lines)
    def _build_obj(self, content, kind, append, new_line):
        self.mime_type = self.dic_kind.get(kind)
        if not self.mime_type:
            return content, False
        if append:
            sep = '\n' if new_line else ''
            self.content = self.limit(self.content + sep + content)
        else:
            self.content = content
        return {self.mime_type: self.content}, True  
    def update(self, content, kind=None, append=False, new_line=True):
        obj, raw = self._build_obj(content, kind, append, new_line)
        self.h.update(obj, raw=raw)
        
out = Output(name='subscribe-status')
show_lines = []

def show(channel, name, value):
    ts = dt.datetime.now().strftime('%H:%M:%S.%f')[:-4]
    s = f'received on {channel}: {name} --> {value} ({ts})'
    out.update(s, kind='text', append=True)


received on top: channel-1 --> {} (12:41:45.75)
received on channel-1: vector2 --> 19 (12:41:54.62)
received on channel-1: vector2 --> 19 (12:42:03.27)
received on channel-1: vector2 --> 19 (12:42:03.46)
received on channel-1: vector2 --> 19 (12:42:03.60)
received on channel-1: vector2 --> 20 (12:42:10.55)

In [5]:
out.display()


received on top: channel-1 --> {} (12:41:45.75)
received on channel-1: vector2 --> 19 (12:41:54.62)
received on channel-1: vector2 --> 19 (12:42:03.27)
received on channel-1: vector2 --> 19 (12:42:03.46)
received on channel-1: vector2 --> 19 (12:42:03.60)
received on channel-1: vector2 --> 20 (12:42:10.55)

In [6]:
ref = s.subscribe(action=show, test=lambda name: name=='vector2')

In [ ]:
s.store['channel-1'].get('vector2')

In [ ]:
s.subscribe_stop(ref)